PGN is a file type used for chess game databases

Sample of PGN (portable game notation) file

In [4]:
open('lichess_db_standard_rated_2013-01.pgn').read()[:1000]

'[Event "Rated Classical game"]\n[Site "https://lichess.org/j1dkb5dw"]\n[White "BFG9k"]\n[Black "mamalak"]\n[Result "1-0"]\n[UTCDate "2012.12.31"]\n[UTCTime "23:01:03"]\n[WhiteElo "1639"]\n[BlackElo "1403"]\n[WhiteRatingDiff "+5"]\n[BlackRatingDiff "-8"]\n[ECO "C00"]\n[Opening "French Defense: Normal Variation"]\n[TimeControl "600+8"]\n[Termination "Normal"]\n\n1. e4 e6 2. d4 b6 3. a3 Bb7 4. Nc3 Nh6 5. Bxh6 gxh6 6. Be2 Qg5 7. Bg4 h5 8. Nf3 Qg6 9. Nh4 Qg5 10. Bxh5 Qxh4 11. Qf3 Kd8 12. Qxf7 Nc6 13. Qe8# 1-0\n\n[Event "Rated Classical game"]\n[Site "https://lichess.org/a9tcp02g"]\n[White "Desmond_Wilson"]\n[Black "savinka59"]\n[Result "1-0"]\n[UTCDate "2012.12.31"]\n[UTCTime "23:04:12"]\n[WhiteElo "1654"]\n[BlackElo "1919"]\n[WhiteRatingDiff "+19"]\n[BlackRatingDiff "-22"]\n[ECO "D04"]\n[Opening "Queen\'s Pawn Game: Colle System, Anti-Colle"]\n[TimeControl "480+2"]\n[Termination "Normal"]\n\n1. d4 d5 2. Nf3 Nf6 3. e3 Bf5 4. Nh4 Bg6 5. Nxg6 hxg6 6. Nd2 e6 7. Bd3 Bd6 8. e4 dxe4 9. Nxe4 Rxh2

In [30]:
import pandas as pd
# garbled mess attempt #1
# class Chess:
#     def __init__(self, path):
#         self.path = path
        
#     def pgn_to_df(self): # Formats the pgn string by removing characters and splitting into list of lists
#                          # then puts some of the key information into a pandas dataframe
#         file = open(self.path)
#         text_list = file.read().replace('"', '').replace('[', '').replace(']', '').replace('Event', '***Event').replace('\n\n', '\n').split('***')
#         formatted = [x for x in text_list if x != '']
#         sized = list(map(lambda x: x.split('\n'), formatted))
#         chesslst = [x for x in sized if len(x) == 17]  # Excludes entries lacking complete data (len(list) = 17 indicates all fields have data)
#         chessDF = [{
#             'Event': x[0][6:],
#             'White': x[2][6:],
#             'Black': x[3][6:],
#             'Result': x[4][7:],
#             'WhiteElo': x[7][9:],
#             'BlackElo': x[8][9:],
#             'GameNotation': x[15]
#         } for x in chesslst]
#         return pd.DataFrame(chessDF)

In [40]:
class Chess:
    def __init__(self, path):
        self.path = path
        
    def pgn_to_csv(self):
        f = open(self.path).read()
        formatted_string = f.replace('[Event', '***[Event')
        split_into_list = formatted_string.replace('\n\n', '\n').split('***')
        blanks_stripped = [x for x in split_into_list if x != '']
        sub_lists = list(map(lambda x: x.split('\n'), blanks_stripped))
        return sub_lists

In [41]:
c = Chess('lichess_db_standard_rated_2013-01.pgn')
c.pgn_to_csv()[:1000][1]

['[Event "Rated Classical game"]',
 '[Site "https://lichess.org/a9tcp02g"]',
 '[White "Desmond_Wilson"]',
 '[Black "savinka59"]',
 '[Result "1-0"]',
 '[UTCDate "2012.12.31"]',
 '[UTCTime "23:04:12"]',
 '[WhiteElo "1654"]',
 '[BlackElo "1919"]',
 '[WhiteRatingDiff "+19"]',
 '[BlackRatingDiff "-22"]',
 '[ECO "D04"]',
 '[Opening "Queen\'s Pawn Game: Colle System, Anti-Colle"]',
 '[TimeControl "480+2"]',
 '[Termination "Normal"]',
 '1. d4 d5 2. Nf3 Nf6 3. e3 Bf5 4. Nh4 Bg6 5. Nxg6 hxg6 6. Nd2 e6 7. Bd3 Bd6 8. e4 dxe4 9. Nxe4 Rxh2 10. Ke2 Rxh1 11. Qxh1 Nc6 12. Bg5 Ke7 13. Qh7 Nxd4+ 14. Kd2 Qe8 15. Qxg7 Qh8 16. Bxf6+ Kd7 17. Qxh8 Rxh8 18. Bxh8 1-0',
 '']

In [12]:
c = Chess('lichess_db_standard_rated_2013-01.pgn')

In [13]:
table = c.pgn_to_df()[['Result', 'GameNotation']]

In [14]:
(table.GameNotation.map(lambda x: x[:5]) == '1. e4').describe()

count     121092
unique         2
top         True
freq       72354
Name: GameNotation, dtype: object

In [15]:
table.Result.map(lambda x: x == '1-0').describe()

count     121092
unique         2
top         True
freq       62000
Name: Result, dtype: object

In [16]:
table

,Result,GameNotation
0,1-0,1. e4 e6 2. d4 b6 3. a3 Bb7 4. Nc3 Nh6 5. Bxh6...
1,1-0,1. d4 d5 2. Nf3 Nf6 3. e3 Bf5 4. Nh4 Bg6 5. Nx...
2,1-0,1. e4 e5 2. Nf3 Nc6 3. Bc4 Nf6 4. Nc3 Bc5 5. a...
3,0-1,1. e4 c6 2. Nc3 d5 3. Qf3 dxe4 4. Nxe4 Nd7 5. ...
4,0-1,1. e4 e6 2. f4 d5 3. e5 c5 4. Nf3 Qb6 5. c3 Nc...
...,...,...
121087,0-1,1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. f3 N...
121088,0-1,1. e4 d5 2. d3 dxe4 3. dxe4 Qxd1+ 4. Kxd1 Nc6 ...
121089,0-1,1. e4 d5 2. exd5 Qxd5 3. Nc3 Qd8 4. Nf3 Nc6 5....
121090,1-0,1. e4 f6 2. Bc4 e6 3. d4 Qe7 4. Nc3 Nc6 5. d5 ...
